In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/all.csv')
df=df[["ESR1.log2.tpm" ,"PGR.log2.tpm" , "IL12RB2","HORMAD1","LEMD1","AGR3","GFRA1","ANTXRL",'NXNL2',"SPATA4",'RNF182',
      "TP53"  , "COL5A1", "GATA3",'LYST',
      "Danaher.Mast.cells" , "Act.CD4"  , "MDSC",'NK cells','Tem.CD8','CD8 T cells',"Danaher.Neutrophils","Danaher.NK.cells","Monocytic lineage" ,'Danaher.Exhausted.CD8','Danaher.CD8.T.cells','Cytotoxic lymphocytes','Danaher.Cytotoxic.cells',
      "Taxane.FinalScore","Swanton.PaclitaxelScore", "Taxane.MitosisScore" ,"Swanton.MitosisScore" ,
                                     'IDO1','TIDE.TAM.M2',"ESC.ssgsea.notnorm","ESC.ssgsea.norm"        
,"GGI.ssgsea.notnorm","resp.pCR"]]

In [ ]:
X=df.iloc[:,0:-1]
y=df.iloc[:,-1]

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
class SelectAtMostKBest(SelectKBest):
    def _check_params(self, X, y):
        if not (self.k == "all" or 0 <= self.k <= X.shape[1]):
            # set k to "all" (skip feature selection), if less than k features are available
            self.k = "all"

In [ ]:
AUC_log=[]
for feature in range(X.shape[1]+1):
    # 用特定特征删除后的特征集
    #X_drop = X.drop(X.columns[feature], axis=1)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    if feature < X.shape[1]:
        X_drop = X.drop(X.columns[feature], axis=1)
    else:
        X_drop = X
        
    scaler = StandardScaler()
    kbest = SelectAtMostKBest(score_func=f_classif)
    logres = LogisticRegression(random_state=1, penalty='elasticnet', solver='saga', max_iter=10000, n_jobs=10,class_weight='balanced')
    pipe = Pipeline(steps=[('scaler', scaler), ('kbest', kbest), ('logres', logres)])
    # Parameter ranges
    param_grid = { 'kbest__k': np.arange(2,X_drop.shape[1],1),
                    'logres__C': np.logspace(-3,3,30),
                    'logres__l1_ratio': np.arange(0.1,1.1,0.1) }
    # Optimisation
    search = RandomizedSearchCV(pipe, param_grid, cv=skf, scoring='roc_auc', return_train_score=True, n_jobs=10, verbose=0, n_iter=1000, random_state=0)

    search.fit(X_drop,y)
    
    best_params = search.best_params_
    best_score = search.best_score_
    AUC_log.append(best_score)
    
    print(best_params,best_score)

{'logres__l1_ratio': 0.1, 'logres__C': 0.18873918221350977, 'kbest__k': 35} 0.8713203463203463
{'logres__l1_ratio': 0.1, 'logres__C': 0.3039195382313198, 'kbest__k': 34} 0.8787337662337663
{'logres__l1_ratio': 0.1, 'logres__C': 0.18873918221350977, 'kbest__k': 35} 0.8710497835497835
{'logres__l1_ratio': 0.1, 'logres__C': 0.3039195382313198, 'kbest__k': 34} 0.8724025974025974
{'logres__l1_ratio': 0.1, 'logres__C': 0.18873918221350977, 'kbest__k': 35} 0.8695346320346321
{'logres__l1_ratio': 0.1, 'logres__C': 0.18873918221350977, 'kbest__k': 35} 0.8767857142857144
{'logres__l1_ratio': 0.1, 'logres__C': 0.3039195382313198, 'kbest__k': 34} 0.8683982683982684
{'logres__l1_ratio': 0.1, 'logres__C': 0.3039195382313198, 'kbest__k': 34} 0.8592532467532468
{'logres__l1_ratio': 0.1, 'logres__C': 0.18873918221350977, 'kbest__k': 35} 0.8712662337662337
{'logres__l1_ratio': 0.1, 'logres__C': 0.3039195382313198, 'kbest__k': 34} 0.8624458874458873
{'logres__l1_ratio': 0.1, 'logres__C': 0.11721022975334